### 1. 인증

#### 1.1. 기본 인증 매커니즘
- 인증은 사용자가 누구인지 확인하는 과정
- 클러이언트와 서버간의 상호작용에서 중요
- HTTP 헤더를 사용하여 인증 정보를 서버에 전달

#### 1.2. 인증을 위한 HTTP 헤더
- HTTP 헤더는 클라이언트와 서버간 데이터 교환시 중요한 역할
- `Authorization` 헤더는 인증 정보를 담고있음
- 예: `Authorization: Basic <인증 정보>`
    - `<인증 정보>` 은 '아이디:비밀번호' 를 Base64로 인코딩한 값

#### 1.3. FastAPI 에서 기본 인증 구현

```python
# main_01.py
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import HTTPBasic, HTTPBasicCredentials

app = FastAPI()
security = HTTPBasic()

def get_current_username(credentials: HTTPBasicCredentials = Depends(security)):
    if credentials.username != "william" or credentials.password != "password":
        raise HTTPException(status_code=401, detail="Unauthorized")
    return credentials.username

@app.get("/users/me")
def read_current_user(username: str = Depends(get_current_username)):
    return {"username": username}

# uvicorn main_01:app --reload
```
- `HTTPBasic`은 기본 인증 방식을 구현
- `get_current_username`함수는 사용자 이름과 비밀번호를 검증
- 잘못된 인증 정보의 경우 HTTP 401 오류 반환
- `/users/me` 엔드포인트는 인증된 사용자만 접근 가능

<img src='../source/img/13/01/01.png' width='80%'>

> curl -X 'GET' "http://127.0.0.1:8000/users/me" -u william:password
FastAPI를 활용한 기본 인증 구현 방법을 통해, 사용자의 요청이 올바른 인증 정보를 갖고 있는지 검증하고, 그에 따라 적절한 응답을 제공할 수 있음

### 2. 세션
1. 세션의 역할
- 웹에서 상태를 유지하는 중요한 방법
- HTTP 프로토콜의 `stateless` 문제를 해결함
2. 작동 원리
- 첫 요청 시 서버는 클라이언트에 고유한 세션 ID를 발급
- 세션 ID는 클라이언트 측 쿠키에 저장됨
- 클라이언트의 재요청 시, 쿠키에 저장된 세션 ID를 서버에 전송
- 서버는 세션 ID를 통해 클라이언트를 식별하고 정보를 저장하거나 불러옴

### 3.FastAPI에서 세션 사용하기
#### 3.1. FastAPI와 Starlette 세션 활용
- FastAPI는 Starlette의 기능을 상속받아 세션 관리 기능을 제공
- `SessionMiddleware`는 Starlette의 미들웨어로, 세션 관리를 수행
#### 3.2. 세션 미들웨어 설정
- FastAPI앱에 `SessionMiddleware`를 추가하여 세션 기능을 활성화
- `secret_key`는 세션 데이터를 암호화 하는데 사용되며, 안전하게 관리 되어야 함

```python
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your_secret_key")
```

#### 3.3. `secret_key` 관리
- 환경변수를 사용하여 안전하게 관리
- 키 길이는 충분히 길고 복잡하게 설정
- 필요에 따라 키 관리 서비스 (AWS Secrets Manager, HashiCorp Vault 등) 을 활용
- 주기적인 키 갱신을 고려

#### 3.4. 세션 데이터 설정 및 조회
- `Request`객체를 사용하여 세션에 데이터를 저장하고 조회

```python
@app.post('/set/')
async def set_session(request: Request):
    request.session["username"] = "william"
    return {"msg": "Session value set"}
```

```python
@app.get("/get/")
async def get_session(request: Request):
    username = request.session.get("username", "Guest")
    return {"username": username}
```

#### 3.5. 세션 가져오기 및 테스트

```python
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your-secret-key")

@app.post("/set/")
async def set_session(request: Request):
    request.session["username"] = "william"
    return {"message": "Session value set"}

@app.get("/get/")
async def get_session(request: Request):
    # 세션에서 username 값을 가져옴. 값이 없으면 "Guest"를 기본값으로 사용
    username = request.session.get("username", "Guest")
    return {"username": username}
```

#### 3.6. curl을 사용한 테스트
1. 세션 설정 요청
- `-c cookies.txt`: 쿠키를 `coockies.txt`파일에 저장
    ```bash
    curl -X 'POST' 'http://127.0.0.1:8000/set/' -i -c cookies.txt
    ```

2. 세션 값 가져오기 요청
- `-b cookies.txt`: `cookies.txt`파일에서 쿠키를 읽어 요청에 포함
    ```bash
    curl -X 'GET' 'http://127.0.0.1:8000/get/' -i -b cookies.txt
    ```

<img src='../source/img/13/05/01.png' width='80%'>

> 쿠키 파일의 구조와 플래그 설명 추가해줘


### 4. FastAPI 세션을 이용한 로그인 예제 및 테스트

1. 세션 미들웨어 설정
- `SessionMiddleware` 를 앱에 추가하여 세션 관리를 활성화
- `secret_key`는 세션 데이터를 암호화 하는데 사용
  
2. 로그인 엔드포인트(`/login/`)
- 사용자가 제공한 `username`과 `password`를 검증
- 유효한 자격 증명의 경우, 사용자 이름을 세션에 저장

3. 대시보드 엔드포인트 (`/dashboard/`)
- 세션에서 사용자 이름을 조회
- 로그인하지 않은 사용자는 접근 불가

```python
from fastapi import FastAPI, Request, HTTPException
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()

# 세션 미들웨어 추가, 'verysecret'는 실제 사용 시 안전한 키로 교체해야 합니다.
app.add_middleware(SessionMiddleware, secret_key="verysecret")

@app.post("/login/")
async def login(request: Request, username: str, password: str):
    if username == "william" and password == "1234":
        request.session["username"] = username
        return {"message": "Successfully logged in"}
    else:
        raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/dashboard/")
async def dashboard(request: Request):
    username = request.session.get("username")
    if not username:
        raise HTTPException(status_code=401, detail="Not authorized")
    return {"message": f"Welcome to the dashboard, {username}"}
```

#### 4.1. 테스트
1. 로그인 테스트
- 로그인을 시도하고 세션 쿠키를 `cookies.txt` 파일에 저장

    ```bash
    curl -X "POST" "http://127.0.0.1:8000/login/?username=william&password=1234" -i -c cookies.txt
    ```

2. 대시보드 접근 테스트
- 로그인 후 대시보드에 접근. 세션 쿠키는 `cookies.txt`에서 가져옴

    ```bash
    curl -X "GET" "http://127.0.0.1:8000/dashboard/" -i -b cookies.txt
    ```

### **1. Authentication**

#### **1.1. Basic Authentication Mechanism**
- Authentication is the process of verifying a user's identity.
- It is critical in client-server interactions.
- Authentication data is sent to the server via HTTP headers.

#### **1.2. HTTP Headers for Authentication**
- HTTP headers play a vital role in data exchange between the client and server.
- The `Authorization` header carries the authentication credentials.
- Example: `Authorization: Basic <credentials>`
  - `<credentials>` is the Base64-encoded string of "username:password".

#### **1.3. Implementing Basic Authentication in FastAPI**

```python
# main_01.py
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import HTTPBasic, HTTPBasicCredentials

app = FastAPI()
security = HTTPBasic()

def get_current_username(credentials: HTTPBasicCredentials = Depends(security)):
    if credentials.username != "william" or credentials.password != "password":
        raise HTTPException(status_code=401, detail="Unauthorized")
    return credentials.username

@app.get("/users/me")
def read_current_user(username: str = Depends(get_current_username)):
    return {"username": username}
# To run: uvicorn main_01:app --reload
```

- **HTTPBasic** implements the basic authentication method.
- The `get_current_username` function validates the username and password.
- If the credentials are incorrect, a 401 Unauthorized error is raised.
- The `/users/me` endpoint is accessible only to authenticated users.

<img src='../source/img/13/01/01.png' width='80%'>

> **Example Command:**  
> `curl -X 'GET' "http://127.0.0.1:8000/users/me" -u william:password`  
>  
> Using this basic authentication setup in FastAPI, the server verifies that the request contains valid authentication data and responds accordingly.

### **2. Session**

#### **2.1. Role of Sessions**
- Sessions are essential for maintaining state in web applications.
- They address the stateless nature of the HTTP protocol.

#### **2.2. How Sessions Work**
- Upon the first request, the server issues a unique session ID to the client.
- The session ID is stored in a client-side cookie.
- On subsequent requests, the cookie containing the session ID is sent to the server.
- The server uses the session ID to identify the client and either store or retrieve session data.

### **3. Using Sessions in FastAPI**

#### **3.1. Utilizing FastAPI with Starlette Sessions**
- FastAPI inherits session management capabilities from Starlette.
- The `SessionMiddleware` provided by Starlette handles session management.

#### **3.2. Configuring Session Middleware**
- Add the `SessionMiddleware` to your FastAPI application to enable session support.
- The `secret_key` is used to encrypt session data and must be kept secure.

```python
# main_02.py
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your_secret_key")
```

#### **3.3. Managing the `secret_key`**
- Use environment variables to securely manage the secret key.
- Ensure that the key is sufficiently long and complex.
- Consider using key management services (e.g., AWS Secrets Manager, HashiCorp Vault) if needed.
- Regularly rotate the key as part of good security practices.

#### **3.4. Setting and Retrieving Session Data**
- Use the `Request` object to store and retrieve data from the session.

```python
# main_02.py
@app.post("/set/")
async def set_session(request: Request):
    request.session["username"] = "william"
    return {"msg": "Session value set"}
```

```python
# main_02.py
@app.get("/get/")
async def get_session(request: Request):
    username = request.session.get("username", "Guest")
    return {"username": username}
```

#### **3.5. Session Retrieval and Testing Example**

```python
# main_02.py
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your-secret-key")

@app.post("/set/")
async def set_session(request: Request):
    request.session["username"] = "william"
    return {"message": "Session value set"}

@app.get("/get/")
async def get_session(request: Request):
    # Retrieve the 'username' from the session. If not found, default to "Guest".
    username = request.session.get("username", "Guest")
    return {"username": username}
# To run: uvicorn main_02:app --reload
```

#### **3.6. Testing Sessions with curl**

1. **Set Session Request**  
   Use the `-c cookies.txt` flag to store cookies in a file.

   ```bash
   curl -X "POST" "http://127.0.0.1:8000/set/" -i -c cookies.txt
   ```

2. **Get Session Request**  
   Use the `-b cookies.txt` flag to send cookies from the file with your request.

   ```bash
   curl -X "GET" "http://127.0.0.1:8000/get/" -i -b cookies.txt
   ```

> **Additional Explanation: Cookie File Structure and Flags**  
> 
> - **Cookie File Structure:**  
>   The cookie file (`cookies.txt`) generated by curl contains details for each cookie, such as:
>   - **Domain:** The domain for which the cookie is valid.
>   - **Flag:** Indicates whether the cookie is valid for subdomains.
>   - **Path:** The URL path for which the cookie is valid.
>   - **Secure:** Indicates if the cookie should only be transmitted over secure connections.
>   - **Expiration:** The expiration time of the cookie in UNIX timestamp format.
>   - **Name:** The name of the cookie.
>   - **Value:** The cookie's value.
>   - Example line:  
>     `example.com    FALSE    /    TRUE    0    SESSIONID    abc123`
> 
> - **Cookie Flags in curl:**  
>   - `-c cookies.txt`: Saves cookies from the server response into `cookies.txt`.
>   - `-b cookies.txt`: Loads cookies from `cookies.txt` and sends them with the request.
>   These flags help maintain session state across multiple requests.

### **4. FastAPI Login Example Using Sessions and Testing**

#### **4.1. Session Middleware Setup**
- Add `SessionMiddleware` to the FastAPI app to enable session management.
- The `secret_key` is used to encrypt session data.

#### **4.2. Login Endpoint (`/login/`)**
- Validates the provided `username` and `password`.
- If the credentials are valid, stores the username in the session.

#### **4.3. Dashboard Endpoint (`/dashboard/`)**
- Retrieves the username from the session.
- Denies access if the user is not logged in.

```python
# main_03.py
from fastapi import FastAPI, Request, HTTPException
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()

# Add session middleware; replace 'verysecret' with a secure key in production.
app.add_middleware(SessionMiddleware, secret_key="verysecret")

@app.post("/login/")
async def login(request: Request, username: str, password: str):
    if username == "william" and password == "1234":
        request.session["username"] = username
        return {"message": "Successfully logged in"}
    else:
        raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/dashboard/")
async def dashboard(request: Request):
    username = request.session.get("username")
    if not username:
        raise HTTPException(status_code=401, detail="Not authorized")
    return {"message": f"Welcome to the dashboard, {username}"}
# To run: uvicorn main_03:app --reload
```

#### **4.4. Testing the Login and Dashboard Endpoints**

1. **Login Test**  
   Attempt to log in and store the session cookie in a file using the `-c cookies.txt` flag:

   ```bash
   curl -X "POST" "http://127.0.0.1:8000/login/?username=william&password=1234" -i -c cookies.txt
   ```

2. **Dashboard Access Test**  
   After logging in, access the dashboard using the stored session cookie with the `-b cookies.txt` flag:

   ```bash
   curl -X "GET" "http://127.0.0.1:8000/dashboard/" -i -b cookies.txt
   ```